In [1]:
!git clone https://github.com/DrishtiShrrrma/codeclarity.git


Cloning into 'codeclarity'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 162 (delta 65), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 3.37 MiB | 4.56 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [27]:
%cd /content/codeclarity/generated_code_summaries/gemma-2-9b-it

/content/codeclarity/generated_code_summaries/gemma-2-9b-it


In [28]:
!ls -la

total 3080
drwxr-xr-x 2 root root   4096 Apr 28 04:12 .
drwxr-xr-x 6 root root   4096 Apr 28 04:12 ..
-rw-r--r-- 1 root root 442068 Apr 28 04:12 go_summary_all_languages_gemma-2-9b-it.json
-rw-r--r-- 1 root root 546182 Apr 28 04:12 javascript_summary_all_languages_gemma-2-9b-it.json
-rw-r--r-- 1 root root 541392 Apr 28 04:12 java_summary_all_languages_gemma-2-9b-it.json
-rw-r--r-- 1 root root 499648 Apr 28 04:12 php_summary_all_languages_gemma-2-9b-it.json
-rw-r--r-- 1 root root 565978 Apr 28 04:12 python_summary_all_languages_gemma-2-9b-it.json
-rw-r--r-- 1 root root 538047 Apr 28 04:12 ruby_summary_all_languages_gemma-2-9b-it.json


In [29]:
!pip install -qqq cohere

In [30]:
import os
import cohere

In [42]:
folder_path = "/content/codeclarity/generated_code_summaries/gemma-2-9b-it/"
output_dir = "evaluations"
os.makedirs(output_dir, exist_ok=True)

In [43]:
co = cohere.ClientV2("Z8VuQPeTvunJmHqe5lN65HaEES0BycC9nkCZ6OPJ")

In [47]:
# Function to call Cohere API with prompt
def query_cohere(prompt):
    response = co.chat(
        model="command-a-03-2025",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.message.content[0].text.strip()

In [51]:
# Helper function to clean response text
def clean_response_text(text):
    if text.startswith("```json"):
        text = text[7:-3].strip()
    elif text.startswith("```"):
        text = text[3:-3].strip()
    return text


In [52]:
def build_comparison_prompt(code, english_summary, target_summary, target_language):
    prompt = f"""
You are a multilingual software expert evaluating two summaries of the same code snippet:
- One summary is written in English (reference).
- The other summary is written in {target_language}.

You are given:

CODE:
{code}

ENGLISH SUMMARY:
{english_summary}

{target_language.upper()} SUMMARY:
{target_summary}

Evaluate each summary based on:
- Accuracy (captures the functionality)
- Completeness (includes all important elements)
- Terminology Fidelity (correct use of programming terms)
- Language Quality (grammar and natural phrasing)

Instructions:
- Score 1–5 for each criterion, for both summaries.
- Compute an overall score (1–5) for each summary.
- Indicate whether the {target_language} summary is better, equal, or worse than the English one.
- Provide a short justification (1–3 sentences).

Respond **only** in this strict JSON format:
{{
  "score_english": {{
    "accuracy": 0,
    "completeness": 0,
    "terminology": 0,
    "language_quality": 0,
    "overall_score": 0
  }},
  "score_non_english": {{
    "accuracy": 0,
    "completeness": 0,
    "terminology": 0,
    "language_quality": 0,
    "overall_score": 0
  }},
  "comparison": "better" | "equal" | "worse",
  "justification": "your explanation here"
}}
"""
    return prompt


In [ ]:
import json

# Main processing loop
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        print(f"\n🗂️ Processing file: {filename}")

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for idx, entry in enumerate(data):
            code = entry.get("code", "")
            english_summary = entry.get("summary_english", "")

            if not code or not english_summary:
                print(f"⚠️ Skipping Entry {idx}: Missing code or English summary")
                continue

            for key, target_summary in entry.items():
                if key.startswith("summary_") and key != "summary_english":
                    target_language = key.replace("summary_", "")

                    print(f"🔎 Evaluating Entry {idx} - Language: {target_language}")

                    prompt = build_comparison_prompt(
                        code=code,
                        english_summary=english_summary,
                        target_summary=target_summary,
                        target_language=target_language
                    )

                    try:
                        response_text = query_cohere(prompt)
                        parsed_response = json.loads(response_text.strip())
                    except json.JSONDecodeError:
                        print(f"❌ Failed to parse JSON for Entry {idx}, Language {target_language}")
                        print(response_text)
                        continue

                    # Save output
                    output = {
                        "source_file": filename,
                        "entry_index": idx,
                        "target_language": target_language,
                        "evaluation": parsed_response
                    }

                    output_filename = f"eval_{filename}_{idx}_{target_language}.json"
                    output_path = os.path.join(output_dir, output_filename)

                    with open(output_path, 'w', encoding='utf-8') as f_out:
                        json.dump(output, f_out, ensure_ascii=False, indent=2)

                    print(f"✅ Saved: {output_filename}")


Streaming output truncated to the last 5000 lines.
{
  "score_english": {
    "accuracy": 5,
    "completeness": 5,
    "terminology": 5,
    "language_quality": 5,
    "overall_score": 5
  },
  "score_non_english": {
    "accuracy": 4,
    "completeness": 3,
    "terminology": 4,
    "language_quality": 4,
    "overall_score": 3.75
  },
  "comparison": "worse",
  "justification": "The English summary is more comprehensive, accurately captures all key aspects of the code, and uses precise technical terminology. The Telugu summary, while accurate in its core explanation, omits some important details (e.g., the role of `ioprogress.ProgressReader` and `ProgressTracker`) and has minor inconsistencies in terminology and flow, making it less complete and clear."
}
```
🔎 Evaluating Entry 40 - Language: spanish
❌ Failed to parse JSON for Entry 40, Language spanish
```json
{
  "score_english": {
    "accuracy": 5,
    "completeness": 5,
    "terminology": 5,
    "language_quality": 5,
    "over